## Discusión: Limitaciones de Potencia Estadística

### Tamaño Muestral y Potencia

**Tamaño muestral actual**: 100 muestras por dataset

**Consideraciones**:

1. **Potencia Estadística Limitada para Diferencias Pequeñas**:
   - Con n=100, la potencia para detectar diferencias pequeñas (efectos pequeños, d < 0.3) es limitada.
   - Para detectar diferencias pequeñas con potencia del 80% (α=0.05), se necesitarían aproximadamente 200-300 muestras.
   - Las diferencias grandes (efectos grandes, d > 0.8) son detectables con n=100.

2. **Tests No Paramétricos (Wilcoxon)**:
   - El test de Wilcoxon es menos potente que tests paramétricos (t-test) cuando los datos son normales.
   - Sin embargo, es más robusto ante violaciones de normalidad y adecuado para muestras pequeñas.
   - Con n=100, el test de Wilcoxon tiene buena potencia para diferencias medianas-grandes.

3. **Corrección por Múltiples Comparaciones**:
   - Se realizan múltiples tests (4 métodos → 6 comparaciones por métrica).
   - Sin corrección (Bonferroni, FDR), aumenta el riesgo de falsos positivos (Type I error).
   - **Recomendación**: Considerar corrección de Bonferroni para análisis más conservadores:
     - α_ajustado = α / número_de_comparaciones
     - Para 6 comparaciones: α_ajustado = 0.05 / 6 ≈ 0.0083

4. **Intervalos de Confianza del 95%**:
   - Los IC95% proporcionan una estimación del rango probable de los valores reales.
   - Con n=100, los IC95% son razonablemente precisos (SEM ≈ σ/√100).
   - Si los IC95% de dos métodos no se solapan, sugiere una diferencia significativa.

### Recomendaciones para Futuros Estudios

1. **Aumentar tamaño muestral a 200-300 muestras** para mejorar la potencia estadística.
2. **Aplicar corrección de Bonferroni o FDR** para controlar el error de tipo I en múltiples comparaciones.
3. **Realizar análisis de potencia a priori** para determinar el tamaño muestral necesario según el tamaño del efecto esperado.
4. **Considerar análisis bayesiano** como alternativa complementaria para comparaciones de métodos.

### Interpretación de Resultados

- **p-valor < 0.05**: Evidencia estadística de diferencia (sin corrección por múltiples comparaciones).
- **IC95% no solapados**: Sugiere diferencia significativa entre métodos.
- **Diferencia de medias pequeña pero significativa**: Puede no ser clínicamente relevante; considerar tamaño del efecto (Cohen's d).
